#Определение сдвига видео по корреляции высот тона аудиозаписей

In [ ]:
# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch                          # Импортируем фреймворк PyTorch
import torchaudio                     # Библиотека для работы с аудио в PyTorch
import torchaudio.transforms as T     # Модуль трансформации(изменение громкости, скорости воспроизведения, изменение тональности и т.д.)
import torchaudio.functional as F     # Модуль для выполнения различных операций с аудиофайлом(вычисление спектрограммы, преобразование мел-частоты и т.д.)

import matplotlib.pyplot as plt       # Модуль для работы с графиками и изображениями
from tqdm import tqdm

In [ ]:
# Загрузка аудио
audio_1 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_1_120s_ffmpeg.wav'
audio_2 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_2_120s_ffmpeg.wav'
audio_3 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_3_120s_ffmpeg.wav'
audio_4 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_4_120s_ffmpeg.wav'

audio_list =[audio_1, audio_2, audio_3, audio_4]

In [ ]:
#@title функция определяющая максимальную корреляцию по высоте тона (pitch)
def audio_cor_pitch(audio_1_path, audio_2_path):
    auidio_cor_dict = {}

    # Загружаем звуковой сигнал и его частоту дискретизации.
    SPEECH_WAVEFORM_1, SAMPLE_RATE_1 = torchaudio.load(audio_1_path)
    SPEECH_WAVEFORM_2, SAMPLE_RATE_2 = torchaudio.load(audio_2_path)

    # Преобразование тензоров в одномерные массивы
    audio_1 = SPEECH_WAVEFORM_1.numpy()[0]  # Берем первый канал
    audio_2 = SPEECH_WAVEFORM_2.numpy()[0]  # Берем первый канал

    # Приведение массивов к одной длине
    min_length = min(len(audio_1), len(audio_2))
    audio_1 = audio_1[:min_length]
    audio_2 = audio_2[:min_length]

    # Параметры для сдвига
    shift_step = int(SAMPLE_RATE_1 / 30)  # Сдвиг на 1/30 секунды в выборках
    max_shift_samples = min_length  # Максимальный сдвиг в выборках
    correlation_coefficients = []

    # Вычисление коэффициента корреляции для каждого сдвига
    for shift in tqdm(range(0, max_shift_samples + 1, shift_step), desc="Вычисление корреляции"):
        # Сдвиг audio_1 назад
        shifted_audio_1 = np.zeros_like(audio_1)

        if shift > 0:  # Если сдвиг больше 0
            shifted_audio_1[shift:] = audio_1[:-shift]
        else:  # Если сдвиг равен 0, просто копируем audio_1
            shifted_audio_1 = audio_1.copy()

        # Обрезка до пересечения
        min_length_shifted = min(len(shifted_audio_1), len(audio_2))
        shifted_audio_1 = shifted_audio_1[:min_length_shifted]
        audio_2_trimmed = audio_2[:min_length_shifted]

        # Вычисление коэффициента корреляции
        correlation = np.corrcoef(shifted_audio_1, audio_2_trimmed)[0, 1]
        correlation_coefficients.append(correlation)

    # Убираем NaN значения
    correlation_coefficients_np = np.array(correlation_coefficients)
    valid_indices = np.where(~np.isnan(correlation_coefficients_np))[0]
    valid_correlations = correlation_coefficients_np[valid_indices]

    # Находим максимальное значение среди ненулевых
    max_index = valid_indices[np.argmax(valid_correlations)]  # Индекс максимального значения
    max_correlation = valid_correlations.max()  # Максимальное значение корреляции
    max_shift = max_index * shift_step  # Соответствующий сдвиг в выборках

    # Преобразование сдвига в секунды
    max_shift_seconds = max_shift / SAMPLE_RATE_1

    # Вывод результатов
    print(f'Сдвиг {audio_1_path} относительно {audio_2_path}')
    print(f"Максимальная корреляция: {max_correlation:.4f}, Время сдвига: {max_shift_seconds:.4f} секунд")
    auidio_cor_dict[max_shift_seconds] = max_correlation

    return auidio_cor_dict

In [ ]:
audio_pitch_dict = []
for i in range(1, len(audio_list)):
    audio_pitch_dict.append(audio_cor_pitch(audio_list[0], audio_list[i]))

Вычисление корреляции: 100%|█████████▉| 3600/3601 [06:50<00:00,  8.93it/s]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]

  c /= stddev[None, :]

Вычисление корреляции: 100%|██████████| 3601/3601 [06:50<00:00,  8.77it/s]


Сдвиг /content/drive/My Drive/video/HEIAN_NIDAN_camera_1_120s_ffmpeg.wav относительно /content/drive/My Drive/video/HEIAN_NIDAN_camera_2_120s_ffmpeg.wav
Максимальная корреляция: 0.0105, Время сдвига: 22.3667 секунд


Вычисление корреляции: 100%|██████████| 3601/3601 [07:05<00:00,  8.47it/s]


Сдвиг /content/drive/My Drive/video/HEIAN_NIDAN_camera_1_120s_ffmpeg.wav относительно /content/drive/My Drive/video/HEIAN_NIDAN_camera_3_120s_ffmpeg.wav
Максимальная корреляция: 0.0144, Время сдвига: 30.7333 секунд


Вычисление корреляции: 100%|██████████| 3601/3601 [07:13<00:00,  8.30it/s]

Сдвиг /content/drive/My Drive/video/HEIAN_NIDAN_camera_1_120s_ffmpeg.wav относительно /content/drive/My Drive/video/HEIAN_NIDAN_camera_4_120s_ffmpeg.wav
Максимальная корреляция: 0.0087, Время сдвига: 47.2667 секунд


In [ ]:
audio_pitch_dict

[{22.366666666666667: 0.010534951817550839},
 {30.733333333333334: 0.014374703665161371},
 {47.266666666666666: 0.008696166186871888}]

In [ ]:
# Извлечение ключей
keys_shift = [list(d.keys())[0] for d in audio_pitch_dict]

In [ ]:
keys_shift

[22.366666666666667, 30.733333333333334, 47.266666666666666]

In [ ]:
# Создаем новый список, начиная с 0
new_keys_shift = [0]
new_keys_shift  = new_keys_shift + keys_shift

In [ ]:
new_keys_shift

[0, 22.366666666666667, 30.733333333333334, 47.266666666666666]

In [ ]:
# Загрузка видео
video_1 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_1_120s_ffmpeg.mp4'
video_2 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_2_120s_ffmpeg.mp4'
video_3 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_3_120s_ffmpeg.mp4'
video_4 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_4_120s_ffmpeg.mp4'

input_video_list =[video_1, video_2, video_3, video_4]

In [ ]:
# Исходяшее видео
video_outpu_1 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_1_sync.mp4'
video_outpu_2 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_2_sync.mp4'
video_outpu_3 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_3_sync.mp4'
video_outpu_4 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_4_sync.mp4'

output_video_list =[video_outpu_1, video_outpu_2, video_outpu_3, video_outpu_4]

In [ ]:
import os
def video_crop (input_video, output_video, time_from):
    # Обрезаем видео с помощью ffmpeg, начиная с 0-й секунды и продолжительностью 1 минута
    os.system(f"ffmpeg -i '{input_video}' -ss {time_from} -t {len_video}-{time_from} -c copy '{output_video}'")
    print (f'Видео {output_video} обрезано от {time_from}')

In [ ]:
import os
import subprocess
import json

def get_video_length(input_video):
    # Используем ffprobe для получения информации о видео
    command = [
        'ffprobe',
        '-v', 'error',
        '-show_entries', 'format=duration',
        '-of', 'json',
        input_video
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    duration_info = json.loads(result.stdout)
    return float(duration_info['format']['duration'])

def video_crop(input_video, output_video, time_from):
    # Получаем длину видео
    len_video = get_video_length(input_video)

    # Вычисляем длительность обрезки
    duration = len_video - time_from

    # Проверяем, что duration не отрицательное
    if duration <= 0:
        print("Ошибка: время обрезки выходит за пределы длины видео.")
        return

    # Обрезаем видео с помощью ffmpeg
    os.system(f"ffmpeg -i '{input_video}' -ss {time_from} -t {duration} -c copy '{output_video}'")
    print(f'Видео {output_video} обрезано от {time_from} до {len_video}')

In [ ]:
#Обрезаем все видео
for i in range (len(input_video_list)):
    video_crop (input_video_list[i], output_video_list[i], new_keys_shift[i])

Видео /content/drive/My Drive/video/HEIAN_NIDAN_camera_1_sync.mp4 обрезано от 0 до 120.029
Видео /content/drive/My Drive/video/HEIAN_NIDAN_camera_2_sync.mp4 обрезано от 22.366666666666667 до 120.002
Видео /content/drive/My Drive/video/HEIAN_NIDAN_camera_3_sync.mp4 обрезано от 30.733333333333334 до 120.024
Видео /content/drive/My Drive/video/HEIAN_NIDAN_camera_4_sync.mp4 обрезано от 47.266666666666666 до 120.024


In [ ]:
!pip install -q moviepy

In [ ]:
!pip install --upgrade -q moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 51.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import librosa
from moviepy.editor import VideoFileClip, clips_array

  if event.key is 'enter':



In [ ]:
# Исходяшее видео
video_outpu_1 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_1_sync.mp4'
video_outpu_2 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_2_sync.mp4'
video_outpu_3 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_3_sync.mp4'
video_outpu_4 = '/content/drive/My Drive/video/HEIAN_NIDAN_camera_4_sync.mp4'

output_video_list =[video_outpu_1, video_outpu_2, video_outpu_3, video_outpu_4]


video_clips = [VideoFileClip(video_outpu_1), VideoFileClip(video_outpu_2), VideoFileClip(video_outpu_3), VideoFileClip(video_outpu_4)]


# Создание плитки из видео
final_video = clips_array([[video_clips[0], video_clips[1]],
                            [video_clips[2], video_clips[3]]])

# Сохранение и вывод финального видео
final_video_path = '/content/drive/My Drive/video/pose/final_video_14.mp4'
final_video.write_videofile(final_video_path, codec='libx264')


Moviepy - Building video /content/drive/My Drive/video/pose/final_video_14.mp4.
MoviePy - Writing audio in final_video_14TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/My Drive/video/pose/final_video_14.mp4



t:  61%|██████    | 2183/3601 [34:05<21:29,  1.10it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/My Drive/video/HEIAN_NIDAN_camera_4_sync.mp4, 6220800 bytes wanted but 0 bytes read,at frame 2182/2183, at time 72.73/72.76 sec. Using the last valid frame instead.
  "stdin": sp.DEVNULL,

t:  81%|████████▏ | 2930/3601 [43:05<05:19,  2.10it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/My Drive/video/HEIAN_NIDAN_camera_2_sync.mp4, 6220800 bytes wanted but 0 bytes read,at frame 2912/2913, at time 97.62/97.64 sec. Using the last valid frame instead.
  "stdin": sp.DEVNULL,



Moviepy - Done !
Moviepy - video ready /content/drive/My Drive/video/pose/final_video_14.mp4
